# Ch20: 例外處理機制 - 詳解範例

本 Notebook 包含 **4 個完整的實務案例**,展示例外處理在實際應用中的最佳實踐。

## 📋 案例清單
1. 使用者輸入驗證系統(重試機制)
2. 檔案處理與資源管理(try-except-else-finally 完整流程)
3. 網路請求錯誤處理(模擬 API 客戶端)
4. 資料庫操作與交易處理(模擬 SQLite)

---

## 案例 1: 使用者輸入驗證系統

### 情境
建立一個安全的使用者輸入處理函式,需要:
- 處理無效的數值輸入 (ValueError)
- 處理使用者中斷 (KeyboardInterrupt)
- 實作重試機制(最多 3 次)
- 提供友善的錯誤訊息

### 要處理的例外
- `ValueError`: 輸入無法轉換成整數
- `KeyboardInterrupt`: 使用者按 Ctrl+C 中斷

### 學習重點
- 多重 except 子句
- 重試機制設計
- 友善的使用者體驗

In [ ]:
def get_age_from_user():
    """
    安全地從使用者取得年齡,含錯誤處理與重試機制
    
    Returns:
        int or None: 有效的年齡值,或 None(失敗/取消)
    """
    max_retries = 3
    
    for attempt in range(max_retries):
        try:
            # 嘗試取得輸入
            age_str = input(f"請輸入您的年齡 (嘗試 {attempt + 1}/{max_retries}): ")
            
            # 轉換成整數(可能拋出 ValueError)
            age = int(age_str)
            
            # 業務邏輯驗證
            if age < 0 or age > 150:
                raise ValueError("年齡必須在 0-150 之間")
            
            # 成功取得有效年齡
            print(f"✅ 成功:您的年齡是 {age} 歲")
            return age
        
        except ValueError as e:
            # 處理無效輸入
            print(f"❌ 輸入錯誤:{e}")
            
            if attempt < max_retries - 1:
                print(f"   您還有 {max_retries - attempt - 1} 次機會\n")
            else:
                print("   超過最大重試次數,操作失敗")
                return None
        
        except KeyboardInterrupt:
            # 處理使用者中斷
            print("\n\n⚠️  使用者取消輸入")
            return None
    
    return None

# 測試(在 Jupyter 中手動執行)
# result = get_age_from_user()
# print(f"最終結果: {result}")

print("函式定義完成。測試時請執行上面註解掉的程式碼。")
print("\n測試建議:")
print("1. 輸入 'abc' → ValueError")
print("2. 輸入 '200' → ValueError (超過範圍)")
print("3. 按 Ctrl+C → KeyboardInterrupt")
print("4. 輸入 '25' → 成功")

In [ ]:
# 進階版:通用的數值驗證函式
def get_validated_number(prompt, min_value=None, max_value=None, max_retries=3):
    """
    通用的數值輸入驗證函式
    
    Args:
        prompt: 提示訊息
        min_value: 最小值(可選)
        max_value: 最大值(可選)
        max_retries: 最大重試次數
    
    Returns:
        float or None: 有效的數值,或 None(失敗)
    """
    for attempt in range(max_retries):
        try:
            value_str = input(f"{prompt} (嘗試 {attempt + 1}/{max_retries}): ")
            value = float(value_str)
            
            # 範圍檢查
            if min_value is not None and value < min_value:
                raise ValueError(f"數值不能小於 {min_value}")
            if max_value is not None and value > max_value:
                raise ValueError(f"數值不能大於 {max_value}")
            
            print(f"✅ 有效輸入: {value}")
            return value
        
        except ValueError as e:
            print(f"❌ 錯誤:{e}")
            if attempt == max_retries - 1:
                print("超過最大重試次數")
                return None
        
        except KeyboardInterrupt:
            print("\n使用者取消")
            return None
    
    return None

# 測試範例
# score = get_validated_number("請輸入分數", min_value=0, max_value=100)
# print(f"分數: {score}")

print("✅ 案例 1 完成:學會設計重試機制與友善的錯誤處理")

---

## 案例 2: 檔案處理與資源管理

### 情境
建立一個簡單的日誌系統,需要:
- 讀取設定檔 (config.txt)
- 寫入日誌檔 (log.txt)
- 處理檔案不存在、權限不足等錯誤
- 確保檔案控制代碼一定被關閉

### 要處理的例外
- `FileNotFoundError`: 檔案不存在
- `PermissionError`: 沒有讀寫權限
- `OSError`: 其他作業系統錯誤

### 學習重點
- try-except-else-finally 完整流程
- 資源清理(檔案關閉)
- 錯誤層級處理(具體→一般)

In [ ]:
# 先建立測試檔案
with open("config.txt", "w", encoding="utf-8") as f:
    f.write("log_level=INFO\n")
    f.write("output_format=text\n")

print("測試檔案建立完成:config.txt")

In [ ]:
class SimpleLogger:
    """
    簡單的日誌系統,展示完整的例外處理流程
    """
    
    def __init__(self, config_file, log_file):
        self.config_file = config_file
        self.log_file = log_file
        self.config = {}
    
    def load_config(self):
        """
        讀取設定檔
        展示:try-except-else-finally
        """
        config_handle = None
        
        try:
            print(f"1. 嘗試開啟設定檔: {self.config_file}")
            config_handle = open(self.config_file, 'r', encoding='utf-8')
            
        except FileNotFoundError:
            # 具體例外:檔案不存在
            print(f"   ❌ 錯誤:設定檔 '{self.config_file}' 不存在")
            print("   使用預設設定")
            self.config = {"log_level": "WARNING", "output_format": "text"}
            return False
        
        except PermissionError:
            # 具體例外:權限不足
            print(f"   ❌ 錯誤:沒有權限讀取 '{self.config_file}'")
            return False
        
        except OSError as e:
            # 一般例外:其他系統錯誤
            print(f"   ❌ 系統錯誤:{e}")
            return False
        
        else:
            # 只在沒有例外時執行
            print("   ✅ 成功開啟設定檔")
            
            # 讀取設定
            for line in config_handle:
                line = line.strip()
                if '=' in line:
                    key, value = line.split('=', 1)
                    self.config[key] = value
            
            print(f"   設定內容: {self.config}")
            return True
        
        finally:
            # 無論如何都執行:清理資源
            print("2. finally: 清理資源...")
            if config_handle is not None:
                config_handle.close()
                print("   設定檔已關閉")
    
    def write_log(self, message):
        """
        寫入日誌訊息
        """
        log_handle = None
        
        try:
            # 以附加模式開啟日誌檔
            log_handle = open(self.log_file, 'a', encoding='utf-8')
            
            # 取得當前時間
            from datetime import datetime
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            # 寫入日誌
            log_entry = f"[{timestamp}] {message}\n"
            log_handle.write(log_entry)
            
        except PermissionError:
            print(f"❌ 錯誤:沒有權限寫入 '{self.log_file}'")
            return False
        
        except OSError as e:
            print(f"❌ 系統錯誤:{e}")
            return False
        
        else:
            print(f"✅ 日誌已寫入: {message}")
            return True
        
        finally:
            if log_handle is not None:
                log_handle.close()
    
    def run(self, messages):
        """
        完整流程:載入設定 → 寫入日誌
        """
        print("=== 開始日誌系統 ===")
        print()
        
        # 載入設定
        self.load_config()
        print()
        
        # 寫入日誌
        print("=== 寫入日誌 ===")
        for msg in messages:
            self.write_log(msg)
        
        print()
        print("=== 日誌系統結束 ===")

# 測試
logger = SimpleLogger("config.txt", "app.log")
logger.run([
    "應用程式啟動",
    "使用者登入成功",
    "資料處理完成"
])

In [ ]:
# 測試:設定檔不存在的情況
logger2 = SimpleLogger("non_existent_config.txt", "app2.log")
logger2.run(["測試訊息"])

In [ ]:
# 查看生成的日誌檔
print("=== app.log 內容 ===")
try:
    with open("app.log", "r", encoding="utf-8") as f:
        print(f.read())
except FileNotFoundError:
    print("日誌檔尚未建立")

print("\n✅ 案例 2 完成:學會 try-except-else-finally 完整流程與資源管理")

---

## 案例 3: 網路請求錯誤處理(模擬)

### 情境
模擬一個 API 客戶端,需要處理各種網路錯誤:
- 連線逾時 (Timeout)
- HTTP 錯誤 (4xx, 5xx)
- 網路中斷 (ConnectionError)
- 實作指數退避重試策略

### 學習重點
- 自訂例外類別(模擬)
- 重試策略(指數退避)
- 不同類型錯誤的處理

In [ ]:
import time
import random

# 模擬的例外類別
class TimeoutError(Exception):
    """連線逾時錯誤"""
    pass

class HTTPError(Exception):
    """HTTP 錯誤"""
    def __init__(self, status_code, message):
        self.status_code = status_code
        super().__init__(message)

class ConnectionError(Exception):
    """網路連線錯誤"""
    pass

# 模擬的 API 客戶端
class MockAPIClient:
    """
    模擬 API 客戶端,隨機產生各種錯誤
    """
    
    def __init__(self, failure_rate=0.7):
        self.failure_rate = failure_rate
        self.request_count = 0
    
    def get(self, url, timeout=5):
        """
        模擬 HTTP GET 請求
        """
        self.request_count += 1
        
        # 模擬網路延遲
        time.sleep(0.1)
        
        # 隨機產生錯誤
        if random.random() < self.failure_rate:
            error_type = random.choice(['timeout', 'http_error', 'connection_error'])
            
            if error_type == 'timeout':
                raise TimeoutError(f"請求 {url} 逾時(超過 {timeout} 秒)")
            elif error_type == 'http_error':
                status = random.choice([400, 404, 500, 503])
                raise HTTPError(status, f"HTTP {status} 錯誤")
            else:
                raise ConnectionError(f"無法連線到 {url}")
        
        # 成功回應
        return {"data": "成功取得資料", "status": 200}

print("API 客戶端類別定義完成")

In [ ]:
def fetch_with_retry(url, max_retries=3, base_delay=1):
    """
    含重試機制與指數退避的 API 請求
    
    Args:
        url: 請求網址
        max_retries: 最大重試次數
        base_delay: 基礎延遲時間(秒)
    
    Returns:
        dict or None: 回應資料或 None(失敗)
    """
    client = MockAPIClient(failure_rate=0.6)  # 60% 失敗率
    
    for attempt in range(max_retries):
        try:
            print(f"\n嘗試 {attempt + 1}/{max_retries}: 請求 {url}")
            
            # 發送請求
            response = client.get(url, timeout=5)
            
            # 成功
            print(f"✅ 成功:{response}")
            return response
        
        except TimeoutError as e:
            # 逾時:可重試
            print(f"⏱️  逾時:{e}")
            
            if attempt < max_retries - 1:
                delay = base_delay * (2 ** attempt)  # 指數退避:1, 2, 4 秒
                print(f"   將在 {delay} 秒後重試...")
                time.sleep(delay)
            else:
                print("   超過最大重試次數")
                return None
        
        except HTTPError as e:
            # HTTP 錯誤:依狀態碼決定是否重試
            print(f"❌ HTTP 錯誤:{e} (狀態碼: {e.status_code})")
            
            if 500 <= e.status_code < 600:
                # 5xx 伺服器錯誤:可重試
                print("   伺服器錯誤,可重試")
                if attempt < max_retries - 1:
                    delay = base_delay * (2 ** attempt)
                    print(f"   將在 {delay} 秒後重試...")
                    time.sleep(delay)
                else:
                    return None
            else:
                # 4xx 客戶端錯誤:不重試
                print("   客戶端錯誤,不重試")
                return None
        
        except ConnectionError as e:
            # 網路錯誤:可重試
            print(f"🔌 連線錯誤:{e}")
            
            if attempt < max_retries - 1:
                delay = base_delay * (2 ** attempt)
                print(f"   將在 {delay} 秒後重試...")
                time.sleep(delay)
            else:
                return None
        
        except Exception as e:
            # 未預期的錯誤:不重試
            print(f"💥 未預期的錯誤:{type(e).__name__} - {e}")
            return None
    
    return None

# 測試
result = fetch_with_retry("https://api.example.com/data", max_retries=5)
print(f"\n最終結果: {result}")

In [ ]:
# 批量請求處理
def batch_fetch(urls, max_retries=3):
    """
    批量請求多個 URL,繼續處理即使部分失敗
    """
    results = {}
    
    for url in urls:
        try:
            print(f"\n處理: {url}")
            result = fetch_with_retry(url, max_retries=max_retries)
            
            if result:
                results[url] = result
            else:
                results[url] = {"error": "請求失敗"}
        
        except Exception as e:
            print(f"處理 {url} 時發生錯誤:{e}")
            results[url] = {"error": str(e)}
    
    return results

# 測試
urls = [
    "https://api.example.com/users",
    "https://api.example.com/posts",
    "https://api.example.com/comments"
]

results = batch_fetch(urls, max_retries=2)

print("\n=== 批量請求結果 ===")
for url, result in results.items():
    print(f"{url}: {result}")

print("\n✅ 案例 3 完成:學會處理網路錯誤與重試策略")

---

## 案例 4: 資料庫操作與交易處理(模擬)

### 情境
模擬 SQLite 資料庫操作,處理:
- 資料庫連線錯誤
- SQL 語法錯誤
- 唯一性限制違反 (IntegrityError)
- 交易的 commit/rollback

### 學習重點
- 資料庫例外處理
- 交易管理(原子性)
- finally 確保資源釋放

In [ ]:
import sqlite3
from datetime import datetime

# 建立測試資料庫
def setup_database(db_file="test_users.db"):
    """
    初始化測試資料庫
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    # 建立資料表
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE NOT NULL,
            email TEXT NOT NULL,
            created_at TEXT
        )
    """)
    
    conn.commit()
    conn.close()
    print(f"資料庫初始化完成: {db_file}")

setup_database()

In [ ]:
class UserDatabase:
    """
    使用者資料庫管理類別,展示完整的例外處理
    """
    
    def __init__(self, db_file):
        self.db_file = db_file
    
    def add_user(self, username, email):
        """
        新增使用者
        展示:交易管理 + IntegrityError 處理
        """
        conn = None
        
        try:
            # 連接資料庫
            print(f"1. 連接資料庫: {self.db_file}")
            conn = sqlite3.connect(self.db_file)
            cursor = conn.cursor()
            
            # 插入資料
            print(f"2. 嘗試新增使用者: {username}")
            created_at = datetime.now().isoformat()
            
            cursor.execute(
                "INSERT INTO users (username, email, created_at) VALUES (?, ?, ?)",
                (username, email, created_at)
            )
            
            # 提交交易
            conn.commit()
            print(f"   ✅ 成功新增使用者 '{username}'")
            return True
        
        except sqlite3.IntegrityError as e:
            # 唯一性限制違反(username 已存在)
            print(f"   ❌ 唯一性錯誤:{e}")
            print(f"   使用者名稱 '{username}' 已存在")
            
            if conn:
                conn.rollback()
                print("   交易已回滾")
            
            return False
        
        except sqlite3.OperationalError as e:
            # SQL 語法錯誤或資料表不存在
            print(f"   ❌ 操作錯誤:{e}")
            
            if conn:
                conn.rollback()
            
            return False
        
        except sqlite3.DatabaseError as e:
            # 其他資料庫錯誤
            print(f"   ❌ 資料庫錯誤:{e}")
            
            if conn:
                conn.rollback()
            
            return False
        
        finally:
            # 確保連線關閉
            print("3. finally: 清理資源...")
            if conn:
                conn.close()
                print("   資料庫連線已關閉\n")
    
    def get_user(self, username):
        """
        查詢使用者
        """
        conn = None
        
        try:
            conn = sqlite3.connect(self.db_file)
            cursor = conn.cursor()
            
            cursor.execute(
                "SELECT * FROM users WHERE username = ?",
                (username,)
            )
            
            result = cursor.fetchone()
            
            if result:
                return {
                    "id": result[0],
                    "username": result[1],
                    "email": result[2],
                    "created_at": result[3]
                }
            else:
                return None
        
        except sqlite3.DatabaseError as e:
            print(f"查詢錯誤:{e}")
            return None
        
        finally:
            if conn:
                conn.close()
    
    def update_email(self, username, new_email):
        """
        更新使用者 email
        展示:交易的原子性(全部成功或全部失敗)
        """
        conn = None
        
        try:
            conn = sqlite3.connect(self.db_file)
            cursor = conn.cursor()
            
            # 檢查使用者是否存在
            cursor.execute("SELECT id FROM users WHERE username = ?", (username,))
            if not cursor.fetchone():
                raise ValueError(f"使用者 '{username}' 不存在")
            
            # 更新 email
            cursor.execute(
                "UPDATE users SET email = ? WHERE username = ?",
                (new_email, username)
            )
            
            # 提交交易
            conn.commit()
            print(f"✅ 成功更新 {username} 的 email 為 {new_email}")
            return True
        
        except ValueError as e:
            print(f"❌ 錯誤:{e}")
            if conn:
                conn.rollback()
            return False
        
        except sqlite3.DatabaseError as e:
            print(f"❌ 資料庫錯誤:{e}")
            if conn:
                conn.rollback()
            return False
        
        finally:
            if conn:
                conn.close()

print("UserDatabase 類別定義完成")

In [ ]:
# 測試資料庫操作
db = UserDatabase("test_users.db")

print("=== 測試 1: 新增使用者 ===")
db.add_user("alice", "alice@example.com")
db.add_user("bob", "bob@example.com")

In [ ]:
print("=== 測試 2: 重複新增(觸發 IntegrityError)===")
db.add_user("alice", "alice2@example.com")  # 應該失敗

In [ ]:
print("=== 測試 3: 查詢使用者 ===")
user = db.get_user("alice")
print(f"查詢結果: {user}")

In [ ]:
print("=== 測試 4: 更新 email ===")
db.update_email("alice", "alice.new@example.com")

# 驗證更新
user = db.get_user("alice")
print(f"更新後的資料: {user}")

In [ ]:
print("=== 測試 5: 更新不存在的使用者 ===")
db.update_email("charlie", "charlie@example.com")  # 應該失敗

print("\n✅ 案例 4 完成:學會資料庫例外處理與交易管理")

---

## 🎉 四個案例完成!

### 總結

您已完成 4 個實務案例:

1. **使用者輸入驗證系統** ✅
   - 重試機制設計
   - ValueError / KeyboardInterrupt 處理
   - 友善的使用者體驗

2. **檔案處理與資源管理** ✅
   - try-except-else-finally 完整流程
   - FileNotFoundError / PermissionError / OSError
   - 確保資源清理

3. **網路請求錯誤處理** ✅
   - 指數退避重試策略
   - TimeoutError / HTTPError / ConnectionError
   - 依錯誤類型決定重試

4. **資料庫操作與交易處理** ✅
   - IntegrityError 處理
   - 交易的 commit / rollback
   - 確保資料一致性

### 核心技能

- ✅ 多重 except 子句的正確使用
- ✅ else 與 finally 的適用場景
- ✅ 例外層級排序(具體→一般)
- ✅ 資源管理與清理
- ✅ 重試機制設計
- ✅ 交易處理(原子性)

### 下一步

- 完成 **03-practice.ipynb** (8 題課堂練習)
- 完成 **04-exercises.ipynb** (12 題課後作業)
- 學習 **Ch21: 自訂例外與 raise**